# Improvements after seeing others' Notebooks

Inspiration largely from Chong Zhen Jie:
- notebook - https://www.kaggle.com/code/chongzhenjie/ecuador-store-sales-global-forecasting-lightgbm/notebook#3.-Model-Training
- eda takeaways (not used here)
    * missed zero vals (leading, trailing 0s, number of 0s)
    * holidays (filtering for region, work days, etc.)
    * store clustering

- modelling takeaways
    * interpolate missing dates (like jan. 01, christmas)
    * dif model for each family
    * different samples of training data --> ensemble models

## What are the main improvements I can make?

- features
    * oil, oil rolling
    * transactions
    * store cluster, type, etc.

    * holidays merged (city, state, national, work) - or maybe we can delete holidays

- modelling
    * train dif model per family
    * zero out predictions if there are some # of trailing 0s
    * train models on dif train periods (can also exclude before 2015), then ensemble
    * try weighted lgbm highlighting end of aug (own idea, not sure if itll help at all)